In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize

from bs4 import BeautifulSoup
import requests 
import time

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from my_functions import *

In [106]:
# pip install geopy

**WRONG ORIGINS**

Nick Drake = not Myanmar

Frank Turner = not Bahrain

# **Locations Wikipedia scraping**

In [104]:
df = pd.read_csv('Datasets/df_ratings.csv')
df.shape

(352768, 5)

## Testing

In [108]:
df.sample(5)

,album_id,artist,title,rating,votes
164060,214002,Die Nerven,Out,4.00,6
80422,98805,Sedition,World of Illusion,3.50,2
106981,133748,Gas Rag,Human Rights,3.79,7
274179,380027,Robin Williamson,Songs For The Calendarium,2.75,2
320012,455180,Swear Jar,Swear Jar II,3.75,2


In [125]:
artists = df['artist'].unique()
len(artists)

9616

In [106]:
artists[4155]

'Life at These Speeds'

In [109]:
artist = "AGENT FRESCO"
name_changed = artist.capitalize.replace(' ', '_')
name_changed_band = artist.capitalize.replace(' ', '_') + ('_(band)')
url = f"https://en.wikipedia.org/wiki/{name_changed_band}"

response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
origin = soup.select('table tr th', class_='infobox-label')[2].text
location = soup.select('table tr td', class_='infobox-data')[1].text
location


AttributeError: 'builtin_function_or_method' object has no attribute 'replace'

In [116]:
artist = "AABSINTHE"
name_changed = artist.title().replace(' ', '_')
name_changed

'Aabsinthe'

In [116]:
# 0 - Less_Than_Jake_(band): Gainesville, Florida, U.S.
# 1 - Sparta_(band): El Paso, Texas
# 2 - Muse_(band): Teignmouth, Devon, England
# 3 - Finch: no location found
# 4 - Queens_of_the_Stone_Age: Seattle, Washington, U.S.
# 5 - Transplants_(band): additional citations - Los Angeles, California, United States[1]
# 6 - Dream_Theater_(band): Boston, Massachusetts, United States
# 7 - A_Perfect_Circle: Los Angeles, California, U.S.
# 8 - Rooney_(band): additional citations - Los Angeles, California, U.S.
# 9 - AFI_(band): Ukiah, California, U.S.
# 10 - Sublime_(band): Long Beach, California, U.S.
# 11 - The_Cure_(band): Crawley, England
# 12 - blink-182: Poway, California, U.S.
# 13 - Weezer_(band): Los Angeles, California, U.S.
# 14 - Deftones_(band): Sacramento, California, U.S.
# 15 - Jimmy_Eat_World: Mesa, Arizona, U.S.
# 16 - NOFX: Los Angeles, California, U.S.
# 17 - The_Mars_Volta: El Paso, Texas, U.S.
# 18 - Black_Sabbath_(band): Birmingham, England
# 19 - Raised_Fist: Luleå, Norrbotten, Sweden

In [143]:
artist = "Sugarcult"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
url = f"https://en.wikipedia.org/wiki/{name_changed}"

response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")

# origin = soup.select('table tr th', class_='infobox-label')[2].text
# # location = soup.select('table tr td', class_='infobox-data')[7].text
# location = soup.select('table tr td', class_='infobox-data')[1].text
# location

table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

table[0].text.split('Origin')[1].split('Genres')[0]
# table[0].text.split(')')[2].split('Genres')[0]
# table[0].text

'Santa Barbara, California, United States'

In [118]:
df.head()

,album_id,artist,album,rating,votes
0,37,Less Than Jake,Losing Streak,3.90,414
1,40,Sparta,Wiretap Scars,3.79,431
2,41,Muse,Absolution,3.99,4411
3,42,Muse,Showbiz,3.50,2181
4,45,Finch,What It Is to Burn,3.69,864


## Changing values

In [127]:
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [122]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [ ]:
df.to_csv('Datasets/df_ratings.csv', index=False)

## Other codes

### **CODE**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[0:10]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                else:
                    

                if 'This article' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'This article' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Less_Than_Jake_(band): Gainesville, Florida, U.S.
1 - Sparta_(band): El Paso, Texas
2 - Muse_(band): Teignmouth, Devon, England
3 - Finch: no location found
4 - Queens_of_the_Stone_Age_(band): Seattle, Washington, U.S.
5 - Transplants_(band): Los Angeles, California, United States[1]
6 - Dream_Theater_(band): Boston, Massachusetts, United States
7 - A_Perfect_Circle_(band): Los Angeles, California, U.S.
8 - Rooney_(band): Los Angeles, California, U.S.
9 - AFI_(band): Ukiah, California, U.S.


In [ ]:
# 107 - Emery_(band): This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources: "Emery" band – news · newspapers · books · scholar · JSTOR (June 2008) (Learn how and when to remove this message)
# 142 - Millencolin_(band): This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources: "Millencolin" – news · newspapers · books · scholar · JSTOR (November 2016) (Learn how and when to remove this message)
# 156 - Sugarcult_(band): This biography of a living person needs additional citations for verification. Please help by adding reliable sources. Contentious material about living persons that is unsourced or poorly sourced must be removed immediately from the article and its talk page, especially if potentially libelous.Find sources: "Sugarcult" – news · newspapers · books · scholar · JSTOR (November 2020) (Learn how and when to remove this message)

### **Testing code for strange cases**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[140:150]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                # else:
                    
                if 'multiple issues' in location:
                    location = soup.select('table tr td', class_='infobox-data')[7].text        
                    print(f'{count} - {name_changed_band}: multiple issues - {location}')
                    origin_list.append(location)
                elif 'additional citations' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: additional citations - {location}')
                    origin_list.append(location)

                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'multiple issues' in location:
                            location = soup.select('table tr td', class_='infobox-data')[7].text        
                            print(f'{count} - {name_changed_band}: multiple issues - {location}')
                            origin_list.append(location)
                        elif 'additional citations' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: additional citations - {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Funeral_for_a_Friend_(band): Bridgend, Wales
1 - Millencolin_(band): multiple issues - Örebro, Sweden
2 - The_Flaming_Lips_(band): Oklahoma City, Oklahoma, U.S.
3 - Feeder_(band): Feeder in 2008
4 - Descendents_(band): Manhattan Beach, California, U.S.
5 - PJ Harvey: no location found
6 - Godsmack_(band): Lawrence, Massachusetts U.S.
7 - Blind_Faith_(band): Ripley, Surrey, England
8 - Van_Halen_(band): Pasadena, California, U.S.
9 - Damageplan_(band): Dallas, Texas, U.S.


## **Simplified code**

In [ ]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv', keep_default_na=False)

print(f"{df_ratings['artist'].nunique()} artists")
print(f"{df_ratings.shape[0]} albums")
print(f"Average of {round(df_ratings.shape[0] / df_ratings['artist'].nunique(), 2)} albums per artist")

131116 artists
352768 albums
Average of 2.69 albums per artist


In [ ]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv', keep_default_na=False)
artists = df_ratings['artist'].unique()

df_coordinates_scrapped = pd.read_csv('Datasets/df_coordinates.csv')

artists_list = []
origin_list = []
count=0
scrapped=0

start_index = 30

for index in artists[start_index:start_index+10]:

    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

        location = table[0].text.split('Origin')[1].split('Genres')[0]
        city = location.split(', ')[0]
        count+=1
        
        try:
            if city in df_coordinates_scrapped['city'].values:
                print(f'{scrapped}/{count} - {name_changed_band}: {location} - city already scrapped')
            else:
                artists_list.append(index)  
                origin_list.append(location)
                scrapped+=1
                print(f'{scrapped}/{count} - {name_changed_band}: {location}')
        except:
                artists_list.append(index)  
                origin_list.append(location)
                scrapped+=1
                print(f'{scrapped}/{count} - {name_changed_band}: {location}')

    except:
        try:
            url = f"https://en.wikipedia.org/wiki/{name_changed}"
            response = requests.get(url).content
            soup = BeautifulSoup(response, "html.parser")
            table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

            try:
                count+=1 
                location = table[0].text.split('Origin')[1].split('Genres')[0]
                city = location.split(', ')[0]
 
                try:
                    artists_list.append(index) 
                    origin_list.append(location)
                    if city in df_coordinates_scrapped['city'].values:
                        print(f'{scrapped}/{count} - {name_changed}: {location} - city already scrapped')
                    else:
                        scrapped+=1
                        print(f'{scrapped}/{count} - {name_changed}: {location}')
                except:
                        scrapped+=1
                        print(f'{scrapped}/{count} - {name_changed}: {location}')

            except:
                location = table[0].text.split(')')[2].split('Genres')[0]
                city = location.split(', ')[0]

                try:
                    artists_list.append(index)
                    origin_list.append(location)
                    if city in df_coordinates_scrapped['city'].values:
                        print(f'{scrapped}/{count} - {name_changed} (individual): {location} - city already scrapped')
                    else:
                        scrapped+=1
                        print(f'{scrapped}/{count} - {name_changed} (individual): {location}')
                except:
                        scrapped+=1
                        print(f'{scrapped}/{count} - {name_changed} (individual): {location}')

        except:
            try:
                count+=1
                url = f"https://es.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

                location = table[0].text.split('Origen\n')[1].split(' Información')[0]
                city = location.split(', ')[0]
  
                try:
                    artists_list.append(index)  
                    origin_list.append(location)
                    if city in df_coordinates_scrapped['city'].values:
                        print(f'{scrapped}/{count} - {name_changed} (español): {location}')
                        print('city already scrapped')
                    else:
                        scrapped+=1
                        print(f'{scrapped}/{count} - {name_changed} (español): {location}')
                except:
                        scrapped+=1
                        print(f'{scrapped}/{count} - {name_changed} (español): {location}')

            except:
                scrapped+=1
                print(f'{scrapped}/{count} - {index}: error')
                artists_list.append(index) 
                origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

0/1 - Metallica_(band): Los Angeles, California, U.S. - city already scrapped
1/2 - Incubus_(band): Calabasas, California, U.S.
2/3 - The_Flower_Kings_(band): Sweden
2/4 - Alice_in_Chains_(band): Seattle, Washington, U.S. - city already scrapped
3/5 - Coheed_and_Cambria_(band): Nyack, New York, U.S.
4/6 - Pantera_(band): Arlington, Texas, U.S.
5/7 - Brand_New_(band): Long Island, New York, U.S.
5/8 - King_Crimson_(band): London, England - city already scrapped
5/9 - Tool_(band): Los Angeles, California, U.S. - city already scrapped
6/10 - Revis_(band): Carbondale, Illinois, US


## **Saving the data into a dataframe**

In [76]:
df_locations = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
df_locations.shape

(9, 2)

In [77]:
df_locations

,artist,origin
0,Raised Fist,"Luleå, Norrbotten, Sweden"
1,Yes,"London, England"
2,Brainiac,"Dayton, Ohio, United States"
3,Rise Against,"Chicago, Illinois, U.S."
4,Joe Satriani,"Westbury, New York, U.S."
5,Iron Maiden,"London, England"
6,Streetlight Manifesto,"New Brunswick, New Jersey, United States"
7,O.A.R.,"Rockville, Maryland, United States"
8,The Beatles,"Liverpool, England"


## **Null values**

In [78]:
df_locations['origin'].isna().sum()

0

In [79]:
# percentage of nulls
print(f'{round(df_locations['origin'].isna().sum() / df_locations.shape[0]*100, 2)} % of nulls')

0.0 % of nulls


In [47]:
df_locations[df_locations['origin'].isna()]

,artist,origin


**Let's search for these bands manually**

In [20]:
# df_ratings[df_ratings['artist']=='Incubus (USA-CA)']

In [45]:
df_locations['origin'] = np.where(df_locations['artist']=='Distorted Penguins', 'Cumberland, MD, United States', df_locations['origin'])

In [46]:
# now there should be no more null values
df_locations['origin'].isna().sum()

0

### Changing df_ratings in case needed

In [ ]:
# df_ratings['artist'] = np.where(df_ratings['artist']=='Incubus (USA-CA)', 'Incubus', df_ratings['artist'])
# df_ratings[df_ratings['artist']=='Incubus (USA-CA)']

,album_id,artist,album,rating,votes


In [ ]:
# df_ratings.to_csv('Datasets/df_ratings.csv', index=False)

## **Create ``city``**

In [80]:
df_locations['city'] = df_locations['origin'].apply(lambda x: str(x).split(', ')[0])
df_locations

,artist,origin,city
0,Raised Fist,"Luleå, Norrbotten, Sweden",Luleå
1,Yes,"London, England",London
2,Brainiac,"Dayton, Ohio, United States",Dayton
3,Rise Against,"Chicago, Illinois, U.S.",Chicago
4,Joe Satriani,"Westbury, New York, U.S.",Westbury
5,Iron Maiden,"London, England",London
6,Streetlight Manifesto,"New Brunswick, New Jersey, United States",New Brunswick
7,O.A.R.,"Rockville, Maryland, United States",Rockville
8,The Beatles,"Liverpool, England",Liverpool


In [81]:
print(f'{df_locations['origin'].nunique()} origins')
print(f'{df_locations['city'].nunique()} cities')

8 origins
8 cities


## **``origin`` cleaning: dots, [ ]...**

In [83]:
df_locations.sort_values('city')

,artist,origin,city
3,Rise Against,"Chicago, Illinois, U.S.",Chicago
2,Brainiac,"Dayton, Ohio, United States",Dayton
8,The Beatles,"Liverpool, England",Liverpool
1,Yes,"London, England",London
5,Iron Maiden,"London, England",London
0,Raised Fist,"Luleå, Norrbotten, Sweden",Luleå
6,Streetlight Manifesto,"New Brunswick, New Jersey, United States",New Brunswick
7,O.A.R.,"Rockville, Maryland, United States",Rockville
4,Joe Satriani,"Westbury, New York, U.S.",Westbury


In [84]:
# replace special characters for spaces
df_locations['origin'] = df_locations['origin'].str.replace('.', '')
df_locations['origin'] = df_locations['origin'].str.replace(r'\[\d+\]', '', regex=True)

# show the df to check it has been changed correctly
df_locations.sort_values('city')

,artist,origin,city
3,Rise Against,"Chicago, Illinois, US",Chicago
2,Brainiac,"Dayton, Ohio, United States",Dayton
8,The Beatles,"Liverpool, England",Liverpool
1,Yes,"London, England",London
5,Iron Maiden,"London, England",London
0,Raised Fist,"Luleå, Norrbotten, Sweden",Luleå
6,Streetlight Manifesto,"New Brunswick, New Jersey, United States",New Brunswick
7,O.A.R.,"Rockville, Maryland, United States",Rockville
4,Joe Satriani,"Westbury, New York, US",Westbury


## **Concat with the df_locations already scrapped**

In [85]:
df_locations_scrapped = pd.read_csv('Datasets/df_locations.csv')
df_locations_scrapped.shape

(19, 3)

In [86]:
df_locations_concat = pd.concat([df_locations_scrapped, df_locations])
df_locations_concat.reset_index(drop=True, inplace=True)
df_locations_concat.shape

(28, 3)

### **Exporting to .csv**

In [88]:
df_locations_concat.to_csv('Datasets/df_locations.csv', index=False)

## **GeoPy**

In [89]:
geolocator = Nominatim(user_agent="music_analysis")

unique_origins = df_locations['origin'].unique()

country_list = []
city_list = []
latitude_list = []
longitude_list = []
address_list = []
count = 0

for origin in unique_origins:
  count+=1
  time.sleep(1)
  location = geolocator.geocode(origin)

  print(f'{count}/{len(unique_origins)} - {location.address}')  

# save the info in lists
  country_list.append(location.address.split(', ')[-1])
  city_list.append(location.address.split(', ')[0])
  latitude_list.append(location.latitude)
  longitude_list.append(location.longitude)
  address_list.append(location.address)

1/8 - Luleå, Luleå kommun, Norrbottens län, 971 28, Sverige
2/8 - London, Greater London, England, United Kingdom
3/8 - Dayton, Montgomery County, Ohio, United States
4/8 - Chicago, Cook County, Illinois, United States
5/8 - Village of Westbury, Town of North Hempstead, Nassau County, New York, United States
6/8 - New Brunswick, Middlesex County, New Jersey, United States
7/8 - Rockville, Montgomery County, Maryland, United States
8/8 - Liverpool, Liverpool City Region, England, United Kingdom


### **Save it into a dataframe**

In [90]:
df = pd.DataFrame({'country': country_list
                               , 'city': city_list
                               , 'origin': unique_origins
                               , 'latitude': latitude_list
                               , 'longitude': longitude_list
                               , 'address': address_list})
df.sort_values(['country', 'longitude'])

,country,city,origin,latitude,longitude,address
0,Sverige,Luleå,"Luleå, Norrbotten, Sweden",65.583119,22.145954,"Luleå, Luleå kommun, Norrbottens län, 971 28, ..."
7,United Kingdom,Liverpool,"Liverpool, England",53.407199,-2.991680,"Liverpool, Liverpool City Region, England, Uni..."
1,United Kingdom,London,"London, England",51.507446,-0.127765,"London, Greater London, England, United Kingdom"
3,United States,Chicago,"Chicago, Illinois, US",41.875562,-87.624421,"Chicago, Cook County, Illinois, United States"
2,United States,Dayton,"Dayton, Ohio, United States",39.758948,-84.191607,"Dayton, Montgomery County, Ohio, United States"
6,United States,Rockville,"Rockville, Maryland, United States",39.081798,-77.151684,"Rockville, Montgomery County, Maryland, United..."
5,United States,New Brunswick,"New Brunswick, New Jersey, United States",40.495137,-74.443925,"New Brunswick, Middlesex County, New Jersey, U..."
4,United States,Village of Westbury,"Westbury, New York, US",40.755656,-73.587627,"Village of Westbury, Town of North Hempstead, ..."


In [29]:
df_locations[df_locations['city']=='[2]Tacoma']

,artist,origin,city


In [ ]:
# save it in a csv file
# df_coordinates.to_csv('Datasets/df_coordinates.csv', index=False)

### **Concat with the previous df**

In [91]:
# import the last df
df_coordinates_scrapped = pd.read_csv('Datasets/df_coordinates.csv')
df_coordinates_scrapped.shape

(15, 6)

In [92]:
df_coordinates_scrapped

,country,city,origin,latitude,longitude,address
0,United States,Gainesville,"Gainesville, Florida, US",29.651968,-82.324985,"Gainesville, Alachua County, Florida, United S..."
1,United States,El Paso,"El Paso, Texas",31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
2,United Kingdom,Teignmouth,"Teignmouth, Devon, England",50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
3,United States,Temecula,"Temecula, California, Estados Unidos",33.494635,-117.147366,"Temecula, Riverside County, California, United..."
4,United States,Seattle,"Seattle, Washington, US",47.603832,-122.330062,"Seattle, King County, Washington, United States"
5,United States,Los Angeles,"Los Angeles, California, United States",34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
6,United States,Boston,"Boston, Massachusetts, United States",42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
7,United States,Ukiah,"Ukiah, California, US",39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un..."
8,United States,Long Beach,"Long Beach, California, US",33.769016,-118.191604,"Long Beach, Los Angeles County, California, Un..."
9,United States,Cumberland,"Cumberland, MD, United States",39.652650,-78.762383,"Cumberland, Allegany County, Maryland, 21502, ..."


In [93]:
# concat with the df of the locations I just got
df_coordinates = pd.concat([df_coordinates_scrapped, df])
df_coordinates.reset_index(drop=True, inplace=True)
df_coordinates.shape

(23, 6)

In [94]:
check_duplicates_coordinates(df_coordinates)

No duplicates


In [95]:
df_coordinates

,country,city,origin,latitude,longitude,address
0,United States,Gainesville,"Gainesville, Florida, US",29.651968,-82.324985,"Gainesville, Alachua County, Florida, United S..."
1,United States,El Paso,"El Paso, Texas",31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
2,United Kingdom,Teignmouth,"Teignmouth, Devon, England",50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
3,United States,Temecula,"Temecula, California, Estados Unidos",33.494635,-117.147366,"Temecula, Riverside County, California, United..."
4,United States,Seattle,"Seattle, Washington, US",47.603832,-122.330062,"Seattle, King County, Washington, United States"
5,United States,Los Angeles,"Los Angeles, California, United States",34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
6,United States,Boston,"Boston, Massachusetts, United States",42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
7,United States,Ukiah,"Ukiah, California, US",39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un..."
8,United States,Long Beach,"Long Beach, California, US",33.769016,-118.191604,"Long Beach, Los Angeles County, California, Un..."
9,United States,Cumberland,"Cumberland, MD, United States",39.652650,-78.762383,"Cumberland, Allegany County, Maryland, 21502, ..."


In [96]:
# save it in a csv file
df_coordinates.to_csv('Datasets/df_coordinates.csv', index=False)

## **Statistics**

In [97]:
df_coordinates['country'].value_counts()

country
United States     17
United Kingdom     5
Sverige            1
Name: count, dtype: int64

### Merge with the previous df

In [98]:
df_locations = pd.read_csv('Datasets/df_locations.csv')
df_locations

,artist,origin,city
0,Less Than Jake,"Gainesville, Florida, US",Gainesville
1,Sparta,"El Paso, Texas",El Paso
2,Muse,"Teignmouth, Devon, England",Teignmouth
3,Finch,"Temecula, California, Estados Unidos",Temecula
4,Queens of the Stone Age,"Seattle, Washington, US",Seattle
5,Transplants,"Los Angeles, California, United States",Los Angeles
6,Dream Theater,"Boston, Massachusetts, United States",Boston
7,A Perfect Circle,"Los Angeles, California, US",Los Angeles
8,Rooney,"Los Angeles, California, US",Los Angeles
9,AFI,"Ukiah, California, US",Ukiah


In [99]:
df_coordinates

,country,city,origin,latitude,longitude,address
0,United States,Gainesville,"Gainesville, Florida, US",29.651968,-82.324985,"Gainesville, Alachua County, Florida, United S..."
1,United States,El Paso,"El Paso, Texas",31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
2,United Kingdom,Teignmouth,"Teignmouth, Devon, England",50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
3,United States,Temecula,"Temecula, California, Estados Unidos",33.494635,-117.147366,"Temecula, Riverside County, California, United..."
4,United States,Seattle,"Seattle, Washington, US",47.603832,-122.330062,"Seattle, King County, Washington, United States"
5,United States,Los Angeles,"Los Angeles, California, United States",34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
6,United States,Boston,"Boston, Massachusetts, United States",42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
7,United States,Ukiah,"Ukiah, California, US",39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un..."
8,United States,Long Beach,"Long Beach, California, US",33.769016,-118.191604,"Long Beach, Los Angeles County, California, Un..."
9,United States,Cumberland,"Cumberland, MD, United States",39.652650,-78.762383,"Cumberland, Allegany County, Maryland, 21502, ..."


In [100]:
# merge with the previous df with the artists
df_locations_merged = pd.merge(df_locations, df_coordinates, on=['origin', 'city'])
df_locations_merged.drop(columns=['origin'], inplace=True)
df_locations_merged.sort_values(['country', 'city'])

,artist,city,country,latitude,longitude,address
15,Raised Fist,Luleå,Sverige,65.583119,22.145954,"Luleå, Luleå kommun, Norrbottens län, 971 28, ..."
14,Black Sabbath,Birmingham,United Kingdom,52.479699,-1.902691,"Birmingham, West Midlands, England, United Kin..."
10,The Cure,Crawley,United Kingdom,51.110344,-0.180109,"Crawley, West Sussex, England, United Kingdom"
22,The Beatles,Liverpool,United Kingdom,53.407199,-2.991680,"Liverpool, Liverpool City Region, England, Uni..."
16,Yes,London,United Kingdom,51.507446,-0.127765,"London, Greater London, England, United Kingdom"
19,Iron Maiden,London,United Kingdom,51.507446,-0.127765,"London, Greater London, England, United Kingdom"
2,Muse,Teignmouth,United Kingdom,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
6,Dream Theater,Boston,United States,42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
18,Rise Against,Chicago,United States,41.875562,-87.624421,"Chicago, Cook County, Illinois, United States"
9,Distorted Penguins,Cumberland,United States,39.652650,-78.762383,"Cumberland, Allegany County, Maryland, 21502, ..."


In [101]:
# save it in a csv file
df_locations_merged.to_csv('Datasets/df_locations_merged.csv', index=False)

### Selenium

In [ ]:
url = "https://en.wikipedia.org/wiki/"

PATH = ("C:/Program Files (x86)/chromedriver.exe")
driver = webdriver.Chrome(PATH)
driver.get(url)

time.sleep(2)

search = driver.select('')

NameError: name 'time' is not defined